In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [3]:
from google.colab import files
uploaded = files.upload()

Saving merged_df2.csv to merged_df2.csv


In [5]:
# Read the CSV file
import io
nba_df = pd.read_csv(io.BytesIO(uploaded['merged_df2.csv']))

nba_df.head()

,Unnamed: 0,GAME_DATE_EST,game_id,home_team,away_team,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,...,book_name_under,book_id_under,total1_under,under_odds,point_average_last10,point_against_average_last10,away_point_average_last10,away_point_against_average_last10,over_successful,under_successful
0,0,9/30/2017,11700001,1610612744,1610612743,2017,102,0.411,0.833,0.121,...,Pinnacle Sports,238,222.5,108,116.226415,104.320755,111.176893,110.534142,0,1
1,1,9/30/2017,11700002,1610612747,1610612750,2017,99,0.441,0.706,0.167,...,Bookmaker,93,216.0,-102,104.435540,111.298407,105.205303,105.693687,0,1
2,2,11/1/2017,21700111,1610612752,1610612745,2017,97,0.433,0.846,0.308,...,Pinnacle Sports,238,215.5,-102,99.916667,107.183333,110.787037,104.050926,1,0
3,3,11/1/2017,21700107,1610612755,1610612737,2017,119,0.465,0.560,0.371,...,5Dimes,19,209.0,-105,105.028571,107.771429,101.091912,104.283088,1,0
4,4,11/1/2017,21700108,1610612764,1610612756,2017,116,0.446,0.800,0.333,...,Heritage,169,228.0,-105,109.972222,104.277778,105.366667,115.866667,1,0


In [22]:
# Drop non-beneficial columns if applicable - over/under successful, pts_comb_actual
nba_df.drop(columns=['under_successful'], inplace=True)

KeyError: ignored

In [7]:
# Determine the number of unique values in each column.
nba_df.nunique()

Unnamed: 0                           1210
GAME_DATE_EST                         198
game_id                              1210
home_team                              30
away_team                              30
SEASON                                  1
PTS_home                               68
FG_PCT_home                           223
FT_PCT_home                           174
FG3_PCT_home                          219
AST_home                               29
REB_home                               37
PTS_away                               68
FG_PCT_away                           225
FT_PCT_away                           181
FG3_PCT_away                          207
AST_away                               33
REB_away                               42
HOME_TEAM_WINS                          2
book_name_over                          9
book_id_over                            9
total1_over                            92
over_odds                              21
PTS_comb_actual                   

In [8]:
# Look at actual game points value counts for binning
nba_df['PTS_comb_actual'].value_counts()

215    34
207    33
205    27
213    27
211    27
       ..
255     1
250     1
158     1
171     1
166     1
Name: PTS_comb_actual, Length: 106, dtype: int64

In [9]:
# Choose a cutoff value and create a list of actual game points to be replaced
# use the variable name `pts_types_to_replace`
pts_types = pd.DataFrame(nba_df['PTS_comb_actual'].value_counts()).reset_index()

pts_types_to_replace = []

for index, row in pts_types.iterrows():
    if row['PTS_comb_actual'] < 10:
        pts_types_to_replace.append(row['index'])
pts_types_to_replace


# Replace in dataframe
for pts_types in pts_types_to_replace:
    nba_df['PTS_comb_actual'] = nba_df['PTS_comb_actual'].replace(pts_types,"Other")

# Check to make sure binning was successful
nba_df['PTS_comb_actual'].value_counts()

Other    189
215       34
207       33
213       27
206       27
205       27
211       27
217       26
192       25
208       25
209       25
223       25
204       24
202       24
218       24
221       24
224       23
201       23
226       23
227       22
225       21
193       21
210       21
233       20
187       20
220       19
212       19
203       19
200       19
214       18
195       17
197       17
231       17
191       16
190       16
196       16
232       15
219       15
234       15
198       14
189       14
228       14
216       14
237       14
241       13
194       13
240       13
229       13
199       13
188       13
230       11
236       11
235       11
242       11
Name: PTS_comb_actual, dtype: int64

In [10]:
# Look at point-average-last10 value counts for binning
# Try other columns
pa_counts = nba_df['over_odds'].value_counts()
pa_counts

-105    555
-103    138
-102    116
-110     66
-104     55
-101     48
-107     43
-108     37
-100     37
 100     26
 102     22
-115     13
-106     13
 101     12
-109      8
 103      8
 104      6
 105      4
-111      1
 106      1
 109      1
Name: over_odds, dtype: int64

In [23]:
# Choose a cutoff value and create a list of YYYY to be replaced total1_over
# use the variable name `yyyy_to_replace`
oo_types = pd.DataFrame(nba_df['total1_over'].value_counts()).reset_index()

oo_to_replace = []

for index, row in oo_types.iterrows():
    if row['total1_over'] < 5:
        oo_to_replace.append(row['index'])
oo_to_replace

# Replace in dataframe
for oo_types in oo_to_replace:
    nba_df['total1_over'] = nba_df['total1_over'].replace(oo_types,"Other")

# Check to make sure binning was successful
nba_df['total1_over'].value_counts()

Other    46
208.0    42
217.0    37
212.0    37
210.0    34
         ..
198.5     6
227.5     6
198.0     6
231.0     6
228.5     5
Name: total1_over, Length: 69, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
nba_df = pd.get_dummies(nba_df)

In [25]:
# Split our preprocessed data into our features and target arrays
y = nba_df['over_successful']
X = nba_df.drop(columns=['over_successful'])

In [26]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow import keras
from tensorflow.keras import layers
num_input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(layers.Dense(units = 100, activation = 'relu', input_dim = num_input_features))

# Second hidden layer
nn.add(layers.Dense(units = 50, activation = 'elu'))

# Output layer
nn.add(layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               38300     
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 43,401
Trainable params: 43,401
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
# Train the model
fit_modelnn = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
29/29 [==============================] - 1s 2ms/step - loss: 0.6850 - accuracy: 0.5843
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8975
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1760 - accuracy: 0.9592
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0882 - accuracy: 0.9868
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0429 - accuracy: 0.9956
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 7/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 9/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 10/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 11/

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.7884 - accuracy: 0.8548 - 208ms/epoch - 21ms/step
Loss: 0.7884035110473633, Accuracy: 0.8547855019569397
